In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from collections import Counter
import random
import pandas as pd
from tqdm import tqdm 
from gensim.models import Word2Vec

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SkipGramDataset(Dataset):
    def __init__(self, sentences, window_size=10, min_count=3):
        self.sentences = sentences
        self.window_size = window_size
        self.vocab = self._build_vocab(min_count)
        self.word2idx = {word: idx for idx, word in enumerate(self.vocab)}
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}
        self.data = self._create_data()

    def _build_vocab(self, min_count):
        word_counts = Counter([word for sentence in self.sentences for word in sentence])
        return [word for word, count in word_counts.items() if count >= min_count]

    def _create_data(self):
        data = []
        for sentence in self.sentences:
            for i, word in enumerate(sentence):
                if word in self.word2idx:
                    for j in range(max(0, i - self.window_size), min(len(sentence), i + self.window_size + 1)):
                        if i != j and sentence[j] in self.word2idx:
                            data.append((self.word2idx[word], self.word2idx[sentence[j]]))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx][0]).to(device), torch.tensor(self.data[idx][1]).to(device)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from collections import Counter
import random
import pandas as pd
from tqdm import tqdm 
from gensim.models import Word2Vec

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SkipGramDataset(Dataset):
    def __init__(self, sentences, window_size=10, min_count=3):
        self.sentences = sentences
        self.window_size = window_size
        self.vocab = self._build_vocab(min_count)
        self.word2idx = {word: idx for idx, word in enumerate(self.vocab)}
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}
        self.data = self._create_data()

    def _build_vocab(self, min_count):
        word_counts = Counter([word for sentence in self.sentences for word in sentence])
        return [word for word, count in word_counts.items() if count >= min_count]

    def _create_data(self):
        data = []
        for sentence in self.sentences:
            for i, word in enumerate(sentence):
                if word in self.word2idx:
                    for j in range(max(0, i - self.window_size), min(len(sentence), i + self.window_size + 1)):
                        if i != j and sentence[j] in self.word2idx:
                            data.append((self.word2idx[word], self.word2idx[sentence[j]]))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx][0]).to(device), torch.tensor(self.data[idx][1]).to(device)


In [2]:
def load_and_preprocess_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    
    words = text.split()
    
    print(f"Total number of words: {len(words)}")
    
    sequence_length = 20  
    sequences = [words[i:i+sequence_length] for i in range(0, len(words), sequence_length)]
    
    print(f"Number of sequences: {len(sequences)}")
    print(f"First sequence: {sequences[0]}")
    
    return sequences

In [3]:
sentences=load_and_preprocess_data('data/text8')

Total number of words: 17005207
First 10 words: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']
Number of sequences: 850261
First sequence: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english']


In [4]:
sentences = sentences[0:25000]

In [5]:
dataset = SkipGramDataset(sentences)

In [6]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [7]:
print (len(dataset))
print (len(dataloader))

466068
14565


In [ ]:
users = pd.read_csv('users.csv')
items = pd.read_csv('items.csv')

In [9]:
class SkipGramModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGramModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        output = self.linear(embeds)
        return output
        
def train_model(dataset, embedding_dim=512, batch_size=32, num_epochs=25, learning_rate=0.005):
    model = SkipGramModel(len(dataset.vocab), embedding_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    for epoch in range(num_epochs):
        total_loss = 0
        for batch_idx, (input_words, target_words) in tqdm(enumerate(dataloader), total=len(dataloader)):
            input_words, target_words = input_words.to(device), target_words.to(device)
            optimizer.zero_grad()
            log_probs = model(input_words)
            loss = criterion(log_probs, target_words)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")
    
    return model

In [10]:
model = train_model(dataset)

100%|█████████████████████████████████████████████████████████████████████████████| 14565/14565 [06:13<00:00, 39.04it/s]


Epoch 1/25, Loss: 5.8998


100%|████████████████████████████████████████████████████████████████████████████| 14565/14565 [02:00<00:00, 120.54it/s]


Epoch 2/25, Loss: 5.6330


100%|█████████████████████████████████████████████████████████████████████████████| 14565/14565 [04:35<00:00, 52.80it/s]


...

Epoch 23/25, Loss: 5.2748


100%|█████████████████████████████████████████████████████████████████████████████| 14565/14565 [05:53<00:00, 41.24it/s]


Epoch 24/25, Loss: 5.2720


100%|█████████████████████████████████████████████████████████████████████████████| 14565/14565 [05:41<00:00, 42.67it/s]

Epoch 25/25, Loss: 5.2691


In [60]:
word_embeddings = model.embeddings.weight.detach().cpu().numpy()

def most_similar(word, top_n=5):
    if word not in dataset.word2idx:
        return []
    word_idx = dataset.word2idx[word]
    word_vec = word_embeddings[word_idx]
    similarities = np.dot(word_embeddings, word_vec) / (np.linalg.norm(word_embeddings, axis=1) * np.linalg.norm(word_vec))
    most_similar = similarities.argsort()[-top_n-1:-1][::-1]
    return [(dataset.idx2word[idx], similarities[idx]) for idx in most_similar if idx != word_idx]


In [61]:
print(most_similar("author"))

[('famous', 0.49498144), ('difference', 0.46292886), ('woman', 0.4597501), ('influential', 0.45435798), ('kingdom', 0.4328415)]


In [50]:
torch.save({
            'model_state_dict': model.state_dict(),
            }, 'checkpoints/initial.pkg')


In [52]:
checkpoint = torch.load('checkpoints/initial.pkg', weights_only=True)

In [58]:
checkpoint['model_state_dict']

OrderedDict([('embeddings.weight',
              tensor([[-0.0334,  0.7723, -0.1541,  ..., -0.8312, -0.8679, -0.1490],
                      [ 0.1979,  0.0738, -0.0707,  ..., -0.6376,  0.0895, -0.0600],
                      [-0.1665, -0.0984,  0.0231,  ...,  0.0484,  0.0600, -0.0972],
                      ...,
                      [-0.0420,  1.2516,  0.3799,  ...,  1.1112,  0.0416,  1.6234],
                      [-0.4737,  0.3162,  0.9902,  ...,  0.9877, -0.2667,  0.8550],
                      [-0.9057,  0.2380,  0.2295,  ...,  0.0347, -0.9295, -0.0289]],
                     device='cuda:0')),
             ('linear.weight',
              tensor([[-0.8265, -0.0731, -0.6963,  ..., -0.0869, -0.9737,  1.0510],
                      [-0.7041, -0.2857, -0.5386,  ..., -0.0798, -0.6669,  1.2450],
                      [-0.6976, -0.2072, -0.5487,  ..., -0.0528, -0.7097,  1.0454],
                      ...,
                      [-0.7761, -0.2400, -0.5212,  ...,  0.3490, -0.9173,  1.4619],

# CBOW dataset
 

In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from collections import Counter
import random
import pandas as pd
from tqdm import tqdm 
from gensim.models import Word2Vec

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [55]:
# Count total tokens in the entire dataset
total_tokens = sum(len(sentence) for sentence in sentences)

print(f"Total number of tokens in the dataset: {total_tokens}")


Total number of tokens in the dataset: 20000


In [79]:
def load_and_preprocess_data(file_path, max_tokens=20000):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # Split text into words
    words = text.split()

    # Limit to the first 2000 tokens
    words = words[:max_tokens]

    # Convert list of words into sequences of length `context_size`
    sequence_length = 20  # Sequence length could be smaller for CBOW
    sequences = [words[i:i + sequence_length] for i in range(0, len(words), sequence_length)]

    return sequences

# Load the data
sentences = load_and_preprocess_data('text8.txt', max_tokens=500000)


In [80]:
class CBOWDataset(Dataset):
    def __init__(self, sentences, min_count=5, context_size=2):
        self.sentences = sentences
        self.context_size = context_size
        self.vocab = self.build_vocab(sentences, min_count)
        self.word_to_idx = {word: i for i, word in enumerate(self.vocab)}
        self.idx_to_word = {i: word for i, word in enumerate(self.vocab)}
        self.data = []

        for sentence in sentences:
            for i in range(context_size, len(sentence) - context_size):
                context = (
                    sentence[i - context_size:i] + sentence[i + 1:i + context_size + 1]
                )
                target = sentence[i]
                
                if all(word in self.word_to_idx for word in context + [target]):
                    self.data.append((context, target))
        
        # Debugging statement to check samples
        print(f"Generated {len(self.data)} samples from {len(sentences)} sentences")

    def build_vocab(self, sentences, min_count):
        word_counts = Counter([word for sentence in sentences for word in sentence])
        return [word for word, count in word_counts.items() if count >= min_count]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context, target = self.data[idx]
        context_idxs = torch.tensor([self.word_to_idx[w] for w in context], dtype=torch.long)
        target_idx = torch.tensor(self.word_to_idx[target], dtype=torch.long)
        return context_idxs, target_idx


In [81]:
class CBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOWModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs):
        # Inputs are context word indices
        embeds = self.embeddings(inputs)
        context_embed = torch.mean(embeds, dim=1)  # Mean of context embeddings
        output = self.linear(context_embed)
        return output


In [82]:
def train_model(dataset, embedding_dim=512, batch_size=32, num_epochs=25, learning_rate=0.005):
    model = CBOWModel(len(dataset.vocab), embedding_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    for epoch in range(num_epochs):
        total_loss = 0
        for batch_idx, (context_words, target_word) in tqdm(enumerate(dataloader), total=len(dataloader)):
            context_words, target_word = context_words.to(device), target_word.to(device)
            optimizer.zero_grad()
            log_probs = model(context_words)
            loss = criterion(log_probs, target_word)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")
    
    return model


In [83]:
def train_cbow_model(dataset, embedding_dim=128, batch_size=32, num_epochs=10, learning_rate=0.005):
    model = CBOWModel(len(dataset.vocab), embedding_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        total_loss = 0
        for batch_idx, (context, target) in tqdm(enumerate(dataloader), total=len(dataloader)):
            context, target = context.to(device), target.to(device)
            optimizer.zero_grad()
            log_probs = model(context)
            loss = criterion(log_probs, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")

    return model

# Train the model
cbow_dataset = CBOWDataset(sentences, min_count=5, context_size=5)
model = train_cbow_model(cbow_dataset)


Generated 117866 samples from 25000 sentences


  5%|▌         | 202/3684 [00:08<02:27, 23.62it/s]


KeyboardInterrupt: 

In [78]:
# Create an inverted dictionary for mapping indices back to words
idx_to_word = {idx: word for word, idx in cbow_dataset.word_to_idx.items()}

# Print out the context and target words for the first few samples
for i in range(5):  # Change 5 to however many samples you'd like to see
    context_indices, target_index = cbow_dataset[i]
    context_words = [idx_to_word[idx.item()] for idx in context_indices]
    target_word = idx_to_word[target_index.item()]
    print(f"Context: {context_words}, Target: {target_word}")



Context: ['anarchism', 'originated', 'a', 'term'], Target: as
Context: ['originated', 'as', 'term', 'of'], Target: a
Context: ['as', 'a', 'of', 'abuse'], Target: term
Context: ['a', 'term', 'abuse', 'first'], Target: of
Context: ['term', 'of', 'first', 'used'], Target: abuse


In [74]:
import torch

def evaluate_model(model, test_sentences, cbow_dataset):
    model.eval()  # Set the model to evaluation mode
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for sentence in test_sentences:
            for i in range(2, len(sentence) - 2):
                # Extract context and target
                context = sentence[i - 2:i] + sentence[i + 1:i + 3]
                target = sentence[i]
                
                # Convert context to indices
                context_indices = [cbow_dataset.word_to_idx[word] for word in context]
                
                # Convert to tensor and reshape for model input
                context_tensor = torch.tensor(context_indices).unsqueeze(0)  # Shape (1, context_size)
                
                # Get the model's prediction
                output = model(context_tensor)
                predicted_idx = torch.argmax(output, dim=1).item()
                
                # Convert the predicted index back to the word
                predicted_word = cbow_dataset.idx_to_word[predicted_idx]
                
                # Compare prediction with target
                if predicted_word == target:
                    correct_predictions += 1
                total_predictions += 1
                
                print(f"Context: {context}, Target: {target}, Predicted: {predicted_word}")

    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    print(f"Accuracy: {accuracy * 100:.2f}%")

# Example sentences to evaluate the model
test_sentences = [
    ['anarchism', 'originated', 'as', 'a', 'term'],
    ['originated', 'as', 'a', 'term', 'of']
]

# Evaluate the model using the test sentences
evaluate_model(model, test_sentences, cbow_dataset)


Context: ['anarchism', 'originated', 'a', 'term'], Target: as, Predicted: have
Context: ['originated', 'as', 'term', 'of'], Target: a, Predicted: abuse
Accuracy: 0.00%


In [76]:
cbow_dataset = CBOWDataset(sentences)
print(f"Vocabulary size: {len(cbow_dataset.vocab)}")
print(f"Number of samples in the dataset: {len(cbow_dataset)}")


Generated 120143 samples from 12500 sentences
Vocabulary size: 5347
Number of samples in the dataset: 120143


In [41]:
cbow_dataset = CBOWDataset(sentences, min_count=1)  # Lower min_count to include more words


In [43]:
for i, sentence in enumerate(sentences[:5]):
    print(f"Sentence {i+1}: {sentence}")



Sentence 1: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english']
Sentence 2: ['revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to']
Sentence 3: ['describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the', 'organization', 'of', 'society', 'it', 'has', 'also', 'been', 'taken', 'up', 'as']
Sentence 4: ['a', 'positive', 'label', 'by', 'self', 'defined', 'anarchists', 'the', 'word', 'anarchism', 'is', 'derived', 'from', 'the', 'greek', 'without', 'archons', 'ruler', 'chief', 'king']
Sentence 5: ['anarchism', 'as', 'a', 'political', 'philosophy', 'is', 'the', 'belief', 'that', 'rulers', 'are', 'unnecessary', 'and', 'should', 'be', 'abolished', 'although', 'there', 'are', 'differing']


# Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming the dataframe is already loaded as 'df'
# If not, you would load it like this:
# df = pd.read_csv('hacker_news_dataset.csv')

# Convert 'time' column to datetime if it's not already
df['time'] = pd.to_datetime(df['time'])

# Extract month from the 'time' column
df['month'] = df['time'].dt.strftime('%B')

# Count submissions for each month
monthly_submissions = df['month'].value_counts().sort_index()

# Define month order for proper sorting
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

# Reindex the series to ensure all months are included and properly ordered
monthly_submissions = monthly_submissions.reindex(month_order).fillna(0)

# Create a bar plot
plt.figure(figsize=(12, 6))
monthly_submissions.plot(kind='bar')
plt.title('Number of Hacker News Submissions per Month')
plt.xlabel('Month')
plt.ylabel('Number of Submissions')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming the dataframe is already loaded as 'df'
# If not, you would load it like this:
# df = pd.read_csv('hacker_news_dataset.csv')

# Convert 'time' column to datetime if it's not already
df['time'] = pd.to_datetime(df['time'])

# Extract year from the 'time' column
df['year'] = df['time'].dt.year

# Count submissions for each year
yearly_submissions = df['year'].value_counts().sort_index()

# Create a bar plot
plt.figure(figsize=(12, 6))
yearly_submissions.plot(kind='bar')
plt.title('Number of Hacker News Submissions per Year')
plt.xlabel('Year')
plt.ylabel('Number of Submissions')
plt.xticks(rotation=0)  # Keeping year labels horizontal
plt.tight_layout()

# Show the plot
plt.show()

# Optional: Print the data
print(yearly_submissions)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming the dataframe is already loaded as 'df'
# If not, you would load it like this:
# df = pd.read_csv('hacker_news_dataset.csv')

# Convert 'time' column to datetime if it's not already
df['time'] = pd.to_datetime(df['time'])

# Extract year and month from the 'time' column
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month

# Group by year and month, and count submissions
monthly_submissions = df.groupby(['year', 'month']).size().reset_index(name='count')

# Sort by year and month
monthly_submissions = monthly_submissions.sort_values(['year', 'month'])

# Create a sequential index for x-axis
monthly_submissions['index'] = range(len(monthly_submissions))

# Create the plot
plt.figure(figsize=(15, 8))

# Get unique years for different colors
years = monthly_submissions['year'].unique()
colors = sns.color_palette("husl", n_colors=len(years))

# Plot each year with a different color
for year, color in zip(years, colors):
    year_data = monthly_submissions[monthly_submissions['year'] == year]
    plt.plot(year_data['index'], year_data['count'], label=str(year), color=color)

# Customize the plot
plt.title('Number of Hacker News Submissions per Month (Colored by Year)')
plt.xlabel('Months (Sequential)')
plt.ylabel('Number of Submissions')

# Customize x-axis ticks to show every 12th month (January of each year)
xticks = monthly_submissions[monthly_submissions['month'] == 1]['index']
xtick_labels = monthly_submissions[monthly_submissions['month'] == 1]['year']
plt.xticks(xticks, xtick_labels, rotation=45)

plt.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming the dataframe is already loaded as 'df'
# If not, you would load it like this:
# df = pd.read_csv('hacker_news_dataset.csv')

# Convert 'time' column to datetime if it's not already
df['time'] = pd.to_datetime(df['time'])

# Extract year and month from the 'time' column
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month

# Group by year and month, and count submissions
monthly_submissions = df.groupby(['year', 'month']).size().reset_index(name='count')

# Sort by year and month
monthly_submissions = monthly_submissions.sort_values(['year', 'month'])

# Create a sequential index for x-axis
monthly_submissions['index'] = range(len(monthly_submissions))

# Create the plot
plt.figure(figsize=(20, 10))

# Get unique years for different colors
years = monthly_submissions['year'].unique()
colors = sns.color_palette("husl", n_colors=len(years))

# Plot bars for each month, colored by year
for year, color in zip(years, colors):
    year_data = monthly_submissions[monthly_submissions['year'] == year]
    plt.bar(year_data['index'], year_data['count'], color=color, width=1, align='edge')

# Customize the plot
plt.title('Number of Hacker News Submissions per Month (Colored by Year)', fontsize=16)
plt.xlabel('Months (Sequential)', fontsize=12)
plt.ylabel('Number of Submissions', fontsize=12)

# Customize x-axis ticks to show every 12th month (January of each year)
xticks = monthly_submissions[monthly_submissions['month'] == 1]['index']
xtick_labels = monthly_submissions[monthly_submissions['month'] == 1]['year']
plt.xticks(xticks, xtick_labels, rotation=45, ha='right')

# Add a legend
handles = [plt.Rectangle((0,0),1,1, color=color) for color in colors]
plt.legend(handles, years, title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming the dataframe is already loaded as 'df'
# If not, you would load it like this:
# df = pd.read_csv('hacker_news_dataset.csv')

# Convert 'time' column to datetime if it's not already
df['time'] = pd.to_datetime(df['time'])

# Extract year and month from the 'time' column
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month

# Group by year and month, and count submissions
monthly_submissions = df.groupby(['year', 'month']).size().reset_index(name='count')

# Pivot the data to have years as columns and months as rows
pivot_data = monthly_submissions.pivot(index='month', columns='year', values='count')

# Create the plot
plt.figure(figsize=(15, 8))

# Plot lines for each year
for year in pivot_data.columns:
    plt.plot(pivot_data.index, pivot_data[year], label=str(year), marker='o', markersize=4)

# Customize the plot
plt.title('Number of Hacker News Submissions by Month and Year', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Number of Submissions', fontsize=12)

# Set x-axis ticks to show month names
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.xticks(range(1, 13), month_names)

# Add grid
plt.grid(True, linestyle='--', alpha=0.7)

# Add legend
plt.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Show the plot
plt.show()